# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. https://www.kaggle.com/c/nlp-getting-started
* The output of this notebook is a csv file with clean and lemmatized text data. 

## Reading and Understanding the dataset

In [561]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauage processing toolkit
from nltk.stem import WordNetLemmatizer  #Used for Lemmatizing the text
from nltk.corpus import wordnet          #Used for POS tagging 
from nltk.corpus import stopwords        #Stopwords to be removed from text


Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [562]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [563]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [564]:
len(train_df.keyword.unique())

222

In [565]:
len(train_df.location.unique())

3342

A glimpse at 50 rows from the 'locations' column selected randomly


In [566]:
random.sample(list((train_df.location.unique())),k=30)


['Australian Capital Territory',
 'Wrex',
 'Emirates',
 "Sharkatraz/Bindle's Cleft, PA",
 'y/e/l',
 "satan's colon",
 'Narnia, Maryland',
 'Oklahoma',
 'ona block w/ my BOY ??',
 'Philippines ',
 'Winnipeg, Manitoba',
 'di langit 7 bidadari (^,^ )',
 "Plain O' Texas",
 '@notoriousD12',
 'Woosley',
 'Fresno, CA',
 "Jakarta/Kuala Lumpur/S'pore",
 'call me peach or sam lo',
 'Leeds, England',
 'Burlington, VT',
 'Santa Maria, CA',
 '#HarleyChick#PJNT#RunBenRun',
 'England,UK,Europe,Sol 3.',
 'Stalybridge, Tameside',
 'District of Gentrification/ DC',
 'Tripsburg, ms.',
 'Lynnfield, MA',
 'Dayton, Ohio',
 'Eastbourne England',
 'Scottsdale, AZ']

A glimpse at 50 rows from the 'keywords' column selected randomly

In [567]:
random.sample(list((train_df.keyword.unique())),k=30)


['desolate',
 'body%20bag',
 'desolation',
 'trouble',
 'fatal',
 'collided',
 'wreckage',
 'engulfed',
 'body%20bagging',
 'deluge',
 'bioterror',
 'blown%20up',
 'fear',
 'weapon',
 'natural%20disaster',
 'burned',
 'earthquake',
 'hellfire',
 'heat%20wave',
 'windstorm',
 'ruin',
 'detonate',
 'nuclear%20reactor',
 'survive',
 'dust%20storm',
 'riot',
 'airplane%20accident',
 'tsunami',
 'screamed',
 'terrorist']

A glimpse at 30 rows from the 'text' column selected randomly

In [568]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

['Boy 11 charged with manslaughter in shooting death of Elijah Walker http://t.co/HUrIIVFDKC',
 'Two air ambulances on scene of serious crash between two cars and lorry in ... - http://t.co/9pFEaQeSki http://t.co/fntG70rnkx | #EMSNe\x89Û_',
 'SANTA CRUZ \x89ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf',
 "@OKgooner hahaha great song. 'Spent 15 years getting loaded. 15 years till his liver exploded. Now what's Bob going to do NOW that he...'",
 "Evacuation drill at work. The fire doors wouldn't open so i got to smash the emergency release glass #feelingmanly",
 'Emergency responders prepare for chemical disaster through Hazmat training. http://t.co/q9zixCi8E6',
 "The grenade sound effect on 'Impossible' just destroyed about 10 other hoes careers ??",
 "This weekend is me and Nathan's birthday weekend so if you want to drown yourself in beer do reckless things and potentially die hmu",
 '@DJJOHNBLazE shout out 

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [569]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [570]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method extracts the hashtags from the tweets. These hashtags play a vital role in interpreting the context of the tweet

In [571]:
def retreive_hashtags(text):
    hash_tag = ''
    hash_tag = re.sub('#','',' '.join(re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', text)))  #retreiving hastags
    return hash_tag

# Applying the method to retreive hastags and add to the hastag column
train_df["Hasthags"] = ''
train_df.Hasthags= train_df.text.apply(retreive_hashtags)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Adds a column with hashtag values
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [572]:
def cleaning_text(text):
    
    text = str(text)
    text = text.lower()    ##lower case
    
    text = re.sub(r'@[A-Za-z]+[A-Za-z0-9-_]+', '',text) #removing any word starting with @   \w
    text = re.sub(r'https|www|http\S+', '', text)  #removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text

The following method converts NLTK tags to wordnet tags which would be used to lemmatize the words in the following method

In [573]:
def wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

The following method is used to lemmatize the sentences in the following order:

* Clean the sentences by calling the cleaning_text method on each sentence.
* Tokenizing the sentence and generating a nltk POS tag for each word in the cleaned sentence.
* Converting the nltk POS tag to Wordnet POS tag by calling wordnet_tag method.
* Removes the stopwords
* Lemmatizes the tokens using the pos tags


In [574]:
lemmatizing = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence = cleaning_text(sentence) #Cleaning the sentence
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  #Tokenizing and tagging each word 
    wordnet_tagged = map(lambda x: (x[0], wordnet_tag(x[1])), nltk_tagged)  # Coverting the NLTK tags to wordnet tag
    
    #Lemmatizing the tagged tokens
    lemmatized_sentence = [] #empty list for lemmatized words
    for word, tag in wordnet_tagged:
        if word not in set(stopwords.words('english')):  #removing stopwords
            if tag is None:                   
                lemmatized_sentence.append(word) #adding the word as it is if POS tag missing
            else:        
                #else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizing.lemmatize(word, tag))  ##lemmatizing the token using the POS tag
    return " ".join(lemmatized_sentence)



In [575]:
# Applying the lemmatizing method to Hashtags column 
train_df.Hasthags= train_df.Hasthags.apply(lemmatize_sentence)

# Applying the lemmatizing method to text column 
train_df.text= train_df.text.apply(lemmatize_sentence)

Let's have a look at the cleaned dataframe

In [579]:
train_df.sample(n=30)

,id,keyword,location,text,target,Hasthags
2614,3752,destruction,nan,crackdown destruction restrict multiplayer cra...,0,
1210,1744,buildings burning,il,infosec rather know firefighter day often run ...,0,infosec
6322,9038,stretcher,florida,restore vinyl siding make look new,0,
2918,4197,drown,inside your webcam stop that,send drown like way u think yes,0,yes
127,183,aftershock,marysville ca,praise god ministry tell like wdyouth biblestudy,0,wdyouth biblestudy
2603,3737,destroyed,nan,hi david saw usa walk war destroy life million...,1,irandeal
1034,1500,body bags,tx,bring body bag tho,0,
5677,8101,rescued,west hollywood,summer summervibes california puppy pitmix res...,0,summervibes california puppy pitmix rescue bri...
5492,7837,quarantine,nan,reddit quarantine offensivecontent,0,
4066,5778,forest fires,bc,forest fire could delay official say could goo...,1,


Saving the cleaned dataframe as csv to be used for Modelling.

In [577]:
train_df.to_csv("Clean_train_data.csv")